In [2]:
import gymnasium as gym
import numpy as np
from gymnasium.spaces import (
    Box,
    MultiDiscrete,
    Discrete
)

from gymnasium.utils import seeding
from typing import (
    Dict,
    Any,
    Tuple
)

In [10]:
class SimEdgeEnv(gym.Env):
    def __init__(self, config: Dict[str, Any]):
        super(SimEdgeEnv, self).__init__()
        # Seed the environment

        self.obs_elements = ["cpu_prediction", "containers_hosts", "hosts_alloc", "hosts_requests", "hosts_resources_util",
                             "sla_violations", "containers_request"]
        self.num_hosts = 6
        self.num_resources = 2
        self.num_containers= 6
        self.prediction_length = 6
        self.seed(config.get('seed'))
        self.obs_elements = config['obs_elements']
        print('######################## SimEdge ~~~~~~~~~~~~~~~~~~~~~~~~')
        self.prediction_length = config['prediction_length']

        self.datacenter = config['datacenter']
        self.observation_space, self.action_space = self._setup_space()
        print(self.observation_space)
        print(self.action_space)


    def seed(self, seed=None):
        np.random.seed(seed)
        self.np_random, seed = seeding.np_random(seed)
        self._env_seed = seed
        self.base_env_seed = seed
        return [seed]

    def step(self, action):
        print("Step function called")
        # # Implement the logic for taking a step in the environment
        # observation = self.observation_space.sample()
        # reward = 0.0  # Example reward
        # terminated = False  # Example done condition
        # truncated =  True
        # info = {}
        # return observation, reward, terminated, truncated, info
        return NotImplementedError

    def reset(self, seed=None, options: dict[str, Any] = None):
        # # Handle seeding if provided
        # if seed is not None:
        #     self.seed(seed)

        # # Return the initial observation
        # observation = self.observation_space.sample()
        # info = {}  # Populate with any relevant info if necessary
        
        # # If using Gymnasium v0.26+, return (observation, info)
        # return observation, info

        return NotImplementedError
    
    def _setup_space(self):
        """
        "obs_elements": ["containers_hosts", "hosts_alloc", "hosts_requests", "hosts_usages"]
        sla_items = ['sla_violations']
        containers_requests = ['']

        ["", "", "", "", 
      "", ""]

        """
        #numuber of elements based on the obs in the observation space
        obs_size = 0
        predictions = ['cpu_predictions']  #                   ## Future CPU predictions of each node
        node_resource_size_items = [
            "hosts_alloc",
            "hosts_requests",
            "hosts_resources_util"
        ]
        node_services_size_items = ['containers_request']     ## CPU and mem request of each container
        containers_hosts = ['containers_hosts']               ## Current placement of containers on nodes
        sla_items = ['sla_violations']                        ## SLA violation rate of each container
        for elm in self.obs_elements:
            if elm in node_resource_size_items:
                obs_size += self.num_hosts * self.num_resources
            elif elm in node_services_size_items:
                obs_size += self.num_containers * self.num_resources
            elif elm in predictions:
                obs_size += self.num_hosts * self.prediction_length
            elif elm in sla_items:
                obs_size += self.num_containers
            elif elm in containers_hosts:
                obs_size += self.num_hosts * self.num_containers

        higher_bound = 20 # TEMP find a cleaner way
        # generate observation and action spaces
        observation_space = Box(
            low=0, high=higher_bound, shape=(obs_size, ),
            dtype=np.float64, seed=self._env_seed)

        # if self.discrete_actions:
        action_space = Discrete(2 * self.datacenter.num_hosts, seed=self._env_seed)

        return observation_space, action_space